In [ ]:
import obspy
from obspy.io.sac.util import get_sac_reftime
import os
import glob
from IPython.display import clear_output

In [ ]:
root_dir = "/media/elemento/Element/NTU_RW/NEFS"
os.listdir(root_dir)

In [ ]:
def do_selectSNR(data_dir, TNoise1, TNoise2, TSignal1, TSignal2, thresh):
    """
    Select data according the SNR > thresh, for certain phase e.g., t1(P) which has been stored in sacheader.
    TNoise1, TNoise2 the time before e.g., t1
    TSignal1 the time before t1 and  TSignal2 the time after t1
    """
    if not os.path.exists(data_dir):
        print("Missing File: %s" % data_dir)
        os.sys.exit()
    if(TNoise1 >= TNoise2 or TSignal1 >= TSignal2):
        print("The window has not been well set, check that!")
        os.sys.exit()
    
    # Iterating over all the events
    for ev in os.listdir(data_dir):
        inpath = os.path.join(data_dir, ev)
        os.chdir(inpath)
        print(ev)
    
        # Open the info file
        f = open("snr.info", "w")
        
        # Iterating over all the SAC files
        for sac in glob.glob("*.SAC.bp"):
            try:
                st = obspy.read(sac)
                T = get_sac_reftime(st[0].stats.sac) + st[0].stats.sac.t1

                # Measure noise level
                tr = st.copy()
                tr.trim(starttime = T+TNoise1, endtime = T+TNoise2, pad=True, fill_value=0)
                Noisamean = sum(abs(tr[0].data))/len(tr[0].data)

                # Measure signal level
                tr = st.copy()
                tr.trim(starttime = T+TSignal1, endtime = T+TSignal2, pad=True, fill_value=0)
                SignMax = max(abs(tr[0].data))

                # Define the signal to noise ratio value
                snr = SignMax/Noisamean

                f.write("{} {}\n".format(snr,sac))

                if (snr < thresh):
                    os.remove(sac)
                    print("Removed {} which has a low SNR: {:.2f}".format(sac,snr))
                    
                clear_output(wait=False)
                    
            except:
                os.remove(sac)
        
        # Close the info file
        f.close()

In [ ]:
# Filter data (Noise values, Signal values and Threshold)
TNoise1 = -80   # Noise window_left before t1
TNoise2 = -30   # Noise window_right before t1
TSignal1 = -20  # Signal window_left before t1
TSignal2 = 30   # Signal window_right after t1
thresh = 3.0    # Threshold to select data

In [ ]:
# Filter data
providers = ["IRIS", "GFZ"]
years = [2000, 2001, 2002, 2003]

for year1 in years:
    year2 = year1 + 1
    for provider in providers:
        print(f"For the range: {year1}-{year2}, and provider: {provider}")
        
        data_sac = f'{root_dir}/Download/{provider}/SAC/{year1}'
        
        # Perform filtering
        do_selectSNR(data_sac, TNoise1, TNoise2, TSignal1, TSignal2, thresh)
        
        clear_output(wait=False)